# The 2D heat equation

$$
\alpha^2 \left( \frac{\partial^2u}{\partial x^2} + \frac{\partial^2u}{\partial y^2}\right) = \frac{\partial u}{\partial t}
$$

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
class PINN(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.net = nn.Sequential(
            nn.Linear(3,64),
            nn.Tanh(),
            nn.Linear(64,64),
            nn.Tanh(),
            nn.Linear(64,1),
        )

    def foward(self,x):
        return self.net(x)